Data Scraping Notebook

In [46]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

# Launch the browser
browser = Browser('chrome')

1. Which states require payment of state taxes on gambling winnings?

 Here’s a table showing the latest information regarding which states tax gambling winnings. Keep in mind that each state that requires state tax on gambling income has its own tax rate and guidelines. In some cases, gambling winnings are subject to state tax except for lottery winnings.

In [28]:
# Set the URL to visit to a variable
url = "https://www.playusa.com/taxes/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [29]:
#find all divs that contain state marker
table = soup.find('table', class_='tablepress-responsive')

In [30]:
print(table)

<table class="tablepress tablepress-id-548 tablepress-responsive" id="tablepress-548">
<thead>
<tr class="row-1 odd">
<th class="column-1">State</th><th class="column-2">State tax on gambling income?</th><th class="column-3"></th>
</tr>
</thead>
<tbody class="row-hover">
<tr class="row-2 even">
<td class="column-1">Alabama</td><td class="column-2">Yes</td><td class="column-3"></td>
</tr>
<tr class="row-3 odd">
<td class="column-1">Alaska</td><td class="column-2">No</td><td class="column-3"></td>
</tr>
<tr class="row-4 even">
<td class="column-1">Arizona</td><td class="column-2">Yes</td><td class="column-3"></td>
</tr>
<tr class="row-5 odd">
<td class="column-1">Arkansas</td><td class="column-2">Yes</td><td class="column-3"></td>
</tr>
<tr class="row-6 even">
<td class="column-1">California</td><td class="column-2">No</td><td class="column-3"></td>
</tr>
<tr class="row-7 odd">
<td class="column-1">Colorado</td><td class="column-2">Yes</td><td class="column-3"></td>
</tr>
<tr class="row-

In [31]:
#Extract state-specific tax data from the table and build a dictionary of the data. 
state_tax = {}

# Find the table element
state_tax_list = []

# Find the table element
#table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if table:
    # Find all rows in the table body
    rows = table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            tax_status = cells[1].text.strip()
            
            # Create a dictionary for the current state
            state_data = {'state': state, 'tax_status': tax_status}
            
            # Append the state data dictionary to the list
            state_tax_list.append(state_data)

# Print the extracted data
for state_data in state_tax_list:
    print(state_data)

{'state': 'Alabama', 'tax_status': 'Yes'}
{'state': 'Alaska', 'tax_status': 'No'}
{'state': 'Arizona', 'tax_status': 'Yes'}
{'state': 'Arkansas', 'tax_status': 'Yes'}
{'state': 'California', 'tax_status': 'No'}
{'state': 'Colorado', 'tax_status': 'Yes'}
{'state': 'Connecticut', 'tax_status': 'Yes'}
{'state': 'Delaware', 'tax_status': 'Yes'}
{'state': 'D.C.', 'tax_status': 'Yes'}
{'state': 'Florida', 'tax_status': 'No'}
{'state': 'Georgia', 'tax_status': 'Yes'}
{'state': 'Hawaii', 'tax_status': 'No'}
{'state': 'Idaho', 'tax_status': 'Yes'}
{'state': 'Illinois', 'tax_status': 'Yes'}
{'state': 'Indiana', 'tax_status': 'Yes'}
{'state': 'Iowa', 'tax_status': 'Yes'}
{'state': 'Kansas', 'tax_status': 'Yes'}
{'state': 'Kentucky', 'tax_status': 'Yes'}
{'state': 'Louisiana', 'tax_status': 'Yes'}
{'state': 'Maine', 'tax_status': 'Yes'}
{'state': 'Maryland', 'tax_status': 'Yes'}
{'state': 'Massachusetts', 'tax_status': 'Yes'}
{'state': 'Michigan', 'tax_status': 'Yes'}
{'state': 'Minnesota', 'tax_s

In [ ]:
# write the data to a a dataframe and a csv file

2. Here’s the running total for sports betting handle and revenue reported to date across all active markets:

In [32]:
# Set the URL to visit to a variable
url = "https://www.legalsportsreport.com/sports-betting/revenue/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [33]:
rev_table = soup.find('table', class_='tablepress-id-Rev_Markets')
print(rev_table)

<table class="tablepress tablepress-id-Rev_Markets" id="tablepress-Rev_Markets"><thead><tr class="row-1 odd"><th class="column-1">Market</th><th class="column-2">Handle</th><th class="column-3">Revenue</th><th class="column-4">Hold</th><th class="column-5">Taxes</th></tr></thead><tbody class="row-hover"><tr class="row-2 even"><td class="column-1">New Jersey</td><td class="column-2">$38,874,252,673</td><td class="column-3">$2,810,406,278</td><td class="column-4">7.2%</td><td class="column-5">$389,084,891</td></tr><tr class="row-3 odd"><td class="column-1">Nevada</td><td class="column-2">$33,520,083,578</td><td class="column-3">$1,885,621,000</td><td class="column-4">5.6%</td><td class="column-5">$127,279,418</td></tr><tr class="row-4 even"><td class="column-1">New York</td><td class="column-2">$25,545,018,522</td><td class="column-3">$2,231,885,589</td><td class="column-4">8.7%</td><td class="column-5">$1,116,577,865</td></tr><tr class="row-5 odd"><td class="column-1">Illinois</td><td c

In [34]:
# Create an empty list to store market data
market_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if rev_table:
    # Find all rows in the table body
    rows = rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            market = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            market_data = {'market': market, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            market_data_list.append(market_data)

# Print the extracted data
for market_data in market_data_list:
    print(market_data)

{'market': 'New Jersey', 'handle': '$38,874,252,673', 'revenue': '$2,810,406,278', 'hold': '7.2%', 'taxes': '$389,084,891'}
{'market': 'Nevada', 'handle': '$33,520,083,578', 'revenue': '$1,885,621,000', 'hold': '5.6%', 'taxes': '$127,279,418'}
{'market': 'New York', 'handle': '$25,545,018,522', 'revenue': '$2,231,885,589', 'hold': '8.7%', 'taxes': '$1,116,577,865'}
{'market': 'Illinois', 'handle': '$24,059,865,704', 'revenue': '$1,962,146,676', 'hold': '8.2%', 'taxes': '$312,219,908'}
{'market': 'Pennsylvania', 'handle': '$22,428,431,601', 'revenue': '$1,816,885,229', 'hold': '8.1%', 'taxes': '$450,711,088'}
{'market': 'Colorado', 'handle': '$12,795,525,493', 'revenue': '$857,637,396', 'hold': '6.7%', 'taxes': '$46,491,000'}
{'market': 'Indiana', 'handle': '$12,751,685,026', 'revenue': '$1,085,683,295', 'hold': '8.5%', 'taxes': '$103,236,082'}
{'market': 'Michigan', 'handle': '$11,067,854,802', 'revenue': '$950,198,739', 'hold': '8.6%', 'taxes': '$48,127,549'}
{'market': 'Virginia', 'h

In [35]:
#create a dataframe from the market_data

#remove $ signs from the data and convert to float

#remove % signs from the data, convert to float, and divide by 100 


3. US sports betting revenue by month

In [36]:
month_rev_table = soup.find('table', class_='tablepress-id-Rev_Months')
print(month_rev_table)

<table class="tablepress tablepress-id-Rev_Months" id="tablepress-Rev_Months"><thead><tr class="row-1 odd"><th class="column-1">Month</th><th class="column-2">Handle</th><th class="column-3">Revenue</th><th class="column-4">Hold</th><th class="column-5">Taxes</th></tr></thead><tbody class="row-hover"><tr class="row-2 even"><td class="column-1">June 2018</td><td class="column-2">$309,695,659</td><td class="column-3">$24,630,915</td><td class="column-4">8.0%</td><td class="column-5">$2,221,650</td></tr><tr class="row-3 odd"><td class="column-1">July 2018</td><td class="column-2">$294,006,820</td><td class="column-3">$8,433,994</td><td class="column-4">2.9%</td><td class="column-5">$926,804</td></tr><tr class="row-4 even"><td class="column-1">August 2018</td><td class="column-2">$359,275,692</td><td class="column-3">$23,625,881</td><td class="column-4">6.6%</td><td class="column-5">$2,524,419</td></tr><tr class="row-5 odd"><td class="column-1">September 2018</td><td class="column-2">$817,

In [37]:
# Create an empty list to store market data
month_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if month_rev_table:
    # Find all rows in the table body
    rows = month_rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            month = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            timeline_data = {'month': month, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            month_data_list.append(timeline_data)

# Print the extracted data
for timeline_data in month_data_list:
    print(timeline_data)

{'month': 'June 2018', 'handle': '$309,695,659', 'revenue': '$24,630,915', 'hold': '8.0%', 'taxes': '$2,221,650'}
{'month': 'July 2018', 'handle': '$294,006,820', 'revenue': '$8,433,994', 'hold': '2.9%', 'taxes': '$926,804'}
{'month': 'August 2018', 'handle': '$359,275,692', 'revenue': '$23,625,881', 'hold': '6.6%', 'taxes': '$2,524,419'}
{'month': 'September 2018', 'handle': '$817,681,310', 'revenue': '$95,358,434', 'hold': '11.7%', 'taxes': '$12,245,073'}
{'month': 'October 2018', 'handle': '$854,014,077', 'revenue': '$45,832,391', 'hold': '5.4%', 'taxes': '$5,217,752'}
{'month': 'November 2018', 'handle': '$996,071,884', 'revenue': '$54,349,323', 'hold': '5.5%', 'taxes': '$6,272,898'}
{'month': 'December 2018', 'handle': '$990,933,059', 'revenue': '$82,266,923', 'hold': '8.3%', 'taxes': '$11,262,831'}
{'month': 'January 2019', 'handle': '$1,002,092,655', 'revenue': '$43,052,327', 'hold': '4.3%', 'taxes': '$6,529,559'}
{'month': 'February 2019', 'handle': '$881,756,490', 'revenue': '

In [38]:
#create a dataframe from the market_data

#remove $ signs from the data and convert to float

#remove % signs from the data, convert to float, and divide by 100 

3. Major Professional sports teams of the US and canada 

In [39]:
# Set the URL to visit to a variable
teams_url = "https://en.wikipedia.org/wiki/Major_professional_sports_teams_of_the_United_States_and_Canada"

# Send the browser to the URL
browser.visit(teams_url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [40]:
teams_table = soup.find('table', class_='jquery-tablesorter')
print(teams_table)

<table class="wikitable sortable jquery-tablesorter" style="text-align:left; font-size:95%">
<thead><tr>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Team
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Venue
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">City
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">State/Province
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">League
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Est.
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Franchise notes
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Ref
</th></tr></thead><tbody>
<tr>
<td><a href="/wiki/Anaheim_Ducks" title="Anaheim Ducks">Anaheim Ducks</

Data did not scrape correctly due to table set up --> pulled data by copying from excel --> if anyone wants to try you are more than welcome to dig in --> current file major_teams_usa.xlsx

In [44]:
team_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if teams_table:
    # Find all rows in the table body
    rows = teams_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            team = cells[0].text.strip()
            state = cells[3].text.strip()
            league = cells[4].text.strip()
            
            # Create a dictionary for the current market
            team_data = {'Team': team, 'State': state,'League': league}
            
            # Append the market data dictionary to the list
            team_list.append(team_data)

# Print the extracted data
for team_data in team_list:
    print(team_data)

{'Team': 'Anaheim Ducks', 'State': 'California', 'League': 'NHL'}
{'Team': 'Arizona Cardinals', 'State': 'Arizona', 'League': 'NFL'}
{'Team': 'Arizona Coyotes', 'State': 'NHL', 'League': '1996'}
{'Team': 'Arizona Diamondbacks', 'State': 'MLB', 'League': '1998'}
{'Team': 'Atlanta Braves', 'State': 'Georgia', 'League': '1966'}
{'Team': 'Atlanta Falcons', 'State': 'NFL', 'League': '1966'}
{'Team': 'Atlanta Hawks', 'State': '1968', 'League': 'Previously the Tri-Cities Blackhawks (1949–1951), Milwaukee Hawks (1951–1955) and St. Louis Hawks (1955–1968).'}
{'Team': 'Atlanta United FC', 'State': '2017', 'League': ''}
{'Team': 'Austin FC', 'State': 'Texas', 'League': '2021'}
{'Team': 'Baltimore Orioles', 'State': 'Maryland', 'League': 'MLB'}
{'Team': 'Baltimore Ravens', 'State': '1996', 'League': "The Baltimore Ravens were created from the corporate structure of the previous season's Cleveland Browns. The NFL considers the Ravens to have been an expansion franchise. See also Cleveland Browns re

5. build a large data scrape loop to get each states data in a dataframe --> see if possible  

4. Add geopify API to state list --> Map Leaflet Set up --> https://www.geoapify.com/boundaries-api

5. Census population data was downloaded directly from the census tabular menu - no need to do an API. - https://data.census.gov/table - https://data.census.gov/table?q=B05003&g=010XX00US$0400000&tid=ACSDT1Y2021.B05003 - 
B05003SEX BY AGE BY NATIVITY AND CITIZENSHIP STATUS

6. Summary by online vs in-person betting by state done via download - https://www.gamingtoday.com/states/ 